#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [107]:
import pandas as pd
import pymongo

In [108]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['kickstarter']

In [109]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
#df_ks.head()

Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [110]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [168]:
#df_ks.isna().sum()
#df_ks.head()
df_ks = df_ks.drop(['ID', 'usd pledged'], axis = 1)
df_ks.tail()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real
149995,Scutes Headphones - A new way to listen to you...,Product Design,Design,USD,2014-11-14,40000,2014-10-14 20:44:37,3664,canceled,43,US,3664.00
149996,My Drunk @$$ Reviews,Webseries,Film & Video,USD,2012-05-18,1000,2012-04-18 04:42:47,0,failed,0,US,0.00
149997,Freedom Planet - High Speed Platform Game,Video Games,Games,USD,2013-02-15,2000,2013-01-16 00:53:32,25472.7,successful,1105,US,25472.69
149998,Clown vs Mime,Webseries,Film & Video,USD,2011-05-27,1000,2011-04-27 01:22:52,76,failed,3,US,76.00
149999,Striker's Cafe & Bakery,Food Trucks,Food,USD,2016-05-02,20000,2016-04-03 08:01:57,29,failed,3,US,29.00


### Importer les données

In [122]:
collection.drop()

In [123]:
collection.insert_many(df_ks.to_dict('records'))

In [170]:
#database.collection_names()

## Question 1  

In [148]:
cur = collection.find().sort([("pledged", -1)]).limit(5)
#list(cur)

## Question 2

In [130]:
cur = collection.count({"state":"successful"})
print(cur)

53040


In [139]:
print(df_ks.shape)
#Ce qui vérifie au moins la cohérence.... ou pas
#Plus on exécute les cellules au dessus, plus le nombre de curseurs augmente

(150000, 14)


## Question 3

In [146]:
cur = database.kickstarter.aggregate([{"$group" : {"_id" : "$category", "nombre dans categorie" : {"$sum" : 1}}}])
#list(cur)

## Question 4

In [174]:
cur=collection.find({"$and":[{"country":"FR"}, {"launched":{"$lt":2016}}]})
print(cur.count())

0


## Question 5

In [157]:
cur=collection.find({"$and":[{"country":"US"}, {"usd_pledged_real" : {"$gte" : 200000}}]})
#list(cur)

## Question 6 

In [172]:
cur = collection.find({"name":{"$all":["Sport"]}})
curcount = cur.count()
print(curcount)

0


In [164]:
##L'idée pourrait etre de diviser les names en strings, et de voir si les strings des noms contiendraient le mot "Sport"